In [ ]:
import erfa
import numpy as np
import astropy.units as u

In [ ]:
# ITRS
site1 = 9.712156 * u.deg
site2 = 52.385639 * u.deg
site3 = 200 

In [ ]:

# ICRS (RA, DEC, proper_motions, parallax, radial velocity)
# star
coor1 = 353.22987757 * u.deg
coor2 = 52.27730247 * u.deg
mu_a_cos = 22.9 # mas/year
mu_a = mu_a_cos / np.cos(coor1) * 1e-3 * u.arcsec / u.year
mu_delta = -2.1 * 1e-3 * u.arcsec / u.year
paral = 23 * 1e-3 * u.arcsec
vrad = 25 # km/s

In [ ]:
# IERS deltaX, deltaY, DUT1
deltaX = 0.038 # mas
deltaY = -0.118 # mas
DUT1 = -0.349535 # s

In [ ]:
# IERS polar motion
xp = 0.259371
yp = 0.415573

In [ ]:
# ICRS (RA, DEC, proper_motions, parallax, radial velocity)
# BCRS -> Baricentric Celestial Reference System
# Correction from space motion
# BCRS (RA, DEC, parallax)

In [ ]:
phpa = 952 # HPa
tc = 18.5 # C
rh = 0.83
wl = 0.55 #mum

In [ ]:
utc1, utc2 = erfa.dtf2d('UTC', 2003,8,28,0,37,38.973810)
tai1, tai2 = erfa.utctai(utc1, utc2)
tt1, tt2 = erfa.taitt(tai1, tai2)
print(utc1, utc2)
print(tt1, tt2)

In [ ]:
# EOPs:  polar motion in radians, UT1-UTC in seconds.
xp = 50.995e-3 * erfa.DAS2R
yp = 376.723e-3 * erfa.DAS2R
dut1 = 155.0675e-3
# Corrections to IAU 2000A CIP (radians). 
dx = 0.269e-3 * erfa.DAS2R
dy = -0.274e-3 * erfa.DAS2R

In [ ]:
# ICRS to CIRS (geocentric observer).
res1 = erfa.atci13(coor1.to(u.rad).value, coor2.to(u.rad).value,
            mu_a.to(u.rad/u.year).value, mu_delta.to(u.rad/u.year).value,
            paral.to(u.arcsec).value, 0, tt1, tt2
            
           )
print(coor1, coor2)
print(np.rad2deg(res1[0]), np.rad2deg(res1[1]))

In [ ]:
# CIRS to ICRS (astrometric).
res2 = erfa.atic13(res1[0], res1[1],
            tt1, tt2
           )
print(coor1, coor2)
print(np.rad2deg(res1[0]), np.rad2deg(res1[1]))
print(np.rad2deg(res2[0]), np.rad2deg(res2[1]))

In [ ]:
# ICRS (astrometric) to CIRS (geocentric observer)
res3 = erfa.atci13(res2[0], res2[1],
           0,0,0,0,tt1, tt2)
print('icrs', coor1, coor2)
print('cirs', np.rad2deg(res1[0]), np.rad2deg(res1[1]))
print('icrs (astrometric)', np.rad2deg(res2[0]), np.rad2deg(res2[1]))
print('cirs', np.rad2deg(res3[0]), np.rad2deg(res3[1]))

In [ ]:
# apparent place
ra = erfa.anp(res3[0] - res3[2])
da = res3[1]
print('geocentric apparent', np.rad2deg(ra), np.rad2deg(da))

In [ ]:
# CIRS to topocentric, no refraction
final1 = erfa.atio13(res3[0], res3[2], utc1, utc2, DUT1,
                    site1.to(u.rad).value, site2.to(u.rad).value, site3,
                     xp, yp, 0,0,0,0.5
                    )
# az, zen, ha
print(final1)

In [ ]:
# ICRS to topocentric, no refraction
final2 = erfa.atco13(coor1.to(u.rad).value, coor2.to(u.rad).value,
            mu_a.to(u.rad/u.year).value, mu_delta.to(u.rad/u.year).value,
            paral.to(u.arcsec).value, 0, 
            utc1, utc2, DUT1,
            site1.to(u.rad).value, site2.to(u.rad).value, site3,
            xp, yp, 0,0,0,0.5
                    )
print(final2)

In [ ]:
from astropy.coordinates import SkyCoord, Distance
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

In [ ]:
c = SkyCoord(ra=coor1, dec=coor2, frame='icrs',
             distance=Distance(parallax=23*u.mas),
             pm_ra_cosdec=mu_a_cos * u.mas / u.year,
             pm_dec=-2.1* u.mas / u.year,
             obstime=Time(2000.0, format='decimalyear'),
            radial_velocity=vrad * u.km / u.s)

In [ ]:
location = EarthLocation(lon=site1, lat=site2, height=site3*u.m)

In [ ]:
obstime=Time("2003-08-26 00:37:38.973810", scale='utc')

In [ ]:
caltaz = c.transform_to(AltAz(obstime=obstime, location=location))

In [ ]:
caltaz.zen.rad

In [ ]:
from astropy.utils.data import clear_download_cache
clear_download_cache()  

In [ ]:
from astropy.utils import iers

In [ ]:
iers_b = iers.IERS_B.open()

In [ ]:
iers_b.ut1_utc(obstime)  

In [ ]:
dat = iers.earth_orientation_table.get()  

In [ ]:
dat

In [ ]:
dat['MJD', 'UT1_UTC', 'UT1Flag', 'PM_x', 'PM_y', 'PolPMFlag'] 

In [ ]:
iers.IERS_A_URL